In [1]:
#TO-DO LIST: finish velocity distribution code, finish equivalent width code, finish find_point error code,
#add for loop, finish silicon ratio code, make polyfit error code,build new t_max dictionary, 
#make intrinsic color vs. equivalent width graph, make all figures for poster

#improvements: better way to determina maxima for silicon ratio and equivalent width


In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np 
import glob as glob
import random
from statistics import mode
from numpy import trapz
from astropy.time import Time
from bs4 import BeautifulSoup
import urllib
import urllib.request
import urllib.response
from astroquery.ned import Ned
c = 299792 #speed of light in km/s

In [3]:
def gsmooth(x_array, y_array, var_y, vexp = .001, nsig = 5.0):
    #returns smoothed flux array
    # if no variance array create a constant one
    if len(var_y) == 0:
        var_y = np.zeros(len(y_array))
        
    for i in range(len(var_y)):
        if var_y[i] == 0:
            var_y[i] = 1E-20
            # var_y[i] = 1E-31
    
    # Output y-array
    new_y = np.zeros(len(x_array), float)
    
    # Loop over y-array elements
    for i in range(len(x_array)):
        
        # Construct a Gaussian of sigma = vexp*x_array[i]
        gaussian = np.zeros(len(x_array), float)
        sigma = vexp*x_array[i]
        
        # Restrict range to +/- nsig sigma
        sigrange = np.nonzero(abs(x_array-x_array[i]) <= nsig*sigma)
        gaussian[sigrange] = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-0.5*((x_array[sigrange]-x_array[i])/sigma)**2)
        
        # Multiply Gaussian by 1 / variance
        W_lambda = gaussian / var_y
        
        # Perform a weighted sum to give smoothed y value at x_array[i]
        W0 = np.sum(W_lambda)
        W1 = np.sum(W_lambda*y_array)
        new_y[i] = W1/W0

    # Return smoothed y-array
    return new_y

In [4]:
def autosmooth(x_array, y_array, var_y):
    new_y_init = gsmooth(x_array, y_array, var_y, .001)
    SNR = np.mean(new_y_init / flux_error)
    
    if SNR < 5:
        vexp_auto = .0025 #temp value, need to fine tune using SNR in excel spreadsheet
    elif 5 <= SNR < 20:
        vexp_auto = .002
    elif 20 <= SNR < 40:
        vexp_auto = .0015
    elif 40 <= SNR < 60:
        vexp_auto = .001
    elif 60 <= SNR < 100:
        vexp_auto = .00075
    else:
        vexp_auto = .0005
    
    new_y = gsmooth(x_array, y_array, var_y, vexp_auto)
    return new_y

In [5]:
def get_SN_name(file):
    SN_name = file.split('-')[0]
    return SN_name

In [6]:
#still needs to be fine-tuned for odd cases (lack of sigfigs)
def find_z(SN_name):
    if SN_name == "sn2007if":
        z = 0.0742
    elif SN_name == "sn2007kf":
        z = 0.0460
    elif SN_name == "sn2007kg":
        z = .0070
    elif SN_name == "sn2007kh":
        z = 0.0500
    elif SN_name == "sn2008cf":
        z = 0.0460
    elif SN_name == "sn2008hm":
        z = 0.0197
    elif SN_name == "sn2009lf":
        z = 0.0450
    elif SN_name == "snf20080522000":
        z = 0.0472
    elif SN_name == "snf20080522011":
        z = 0.0397
    elif SN_name == "PTF10bjs":
        z = 0.0300
    
    else:
        query_url = 'https://ned.ipac.caltech.edu/cgi-bin/objsearch?objname=sn2007fs&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES'
        sn_url = query_url.replace("sn2007fs", SN_name)
        response = urllib.request.urlopen(sn_url)
        ned_html = response.read()
        soup = BeautifulSoup(ned_html, 'html.parser')
        ned_text = soup.get_text().split('\n')
        for i in range(len(ned_text)):
            if 'CROSS-IDENTIFICATIONS' in ned_text[i]:
                break
        cross_id_text = ned_text[i:i+5]
        for line in cross_id_text:
            if ':' in line:
                cut1 = line.split('SN')
                for el in cut1:
                    if ':' in el:
                        galaxy = el.split(':')[0].strip()
                        data = Ned.get_table(galaxy, table = 'redshifts')
                        tablelength = len(data)
                        z = data[0][4]
    return z

In [65]:
def build_peak_dict(files):
    
    for file in files:
        with open(file) as f:
            lines = f.readlines()

            peak_mjd_dict = {}
            for line in lines:
                l = line.split()
                if len(l) == 30 and l[0] == 'SN:':
                    peak_mjd_dict[l[1].lower()] = float(l[14])
    
    return peak_mjd_dict

files = glob.glob("lc_fits.fitres")
peak_mjd_dict = build_peak_dict(files)
#print(peak_mjd_dict)


#check which I have both sets of data for
file_list = glob.glob("*.flm")
names_list = []
for a in spectrum_list:
    names_list.append(get_SN_name(a))

for SN in SN_list:
    if SN[2:] in peak_mjd_dict:
        if SN in names_list:
            print (SN)
        

sn2007jg
sn2007nq
sn2008hv
sn2009le


In [53]:
glob.glob("lc_fits_test.docx")

['lc_fits_test.docx']

In [8]:
class supernova:
    def __init__(self, name, z):
        self.name = name
        self.spectra = []
        self.z = z

In [9]:
SN_list = []
file = glob.glob("CfA4_info.txt")[0]
sn_datatext = open(file, "r")
sn_datainfo = sn_datatext.readlines()
start_line = 60
for line in sn_datainfo[start_line:]:
    line_elems = line.split()
    temp = line_elems[0]
    if temp[0] == "2":
        name = "sn" + temp
    else:
        name = temp
    SN_list.append(name)
    
#create array of all objects with fields of supernova class populated
spectra_list = []
z_list = []

print ("Getting redshifts from NED and making supernova objects...")
for SN in SN_list:
    associated_z = find_z(SN)
    z_list.append(associated_z)
    
SN_object_list = []
for a in range(0, len(SN_list)):    
    SN_object = supernova(SN_list[a], z_list[a])
    SN_object_list.append(SN_object)
    print (SN_object.name)

Getting redshifts from NED and making supernova objects...
sn2006ct
sn2006ou
sn2007A
sn2007aj
sn2007bj
sn2007cb
sn2007cc
sn2007cf
sn2007cn
sn2007cs
sn2007ev
sn2007fb
sn2007fq
sn2007fs
sn2007hg
sn2007hj
sn2007hu
sn2007if
sn2007ir
sn2007is
sn2007jg
sn2007kd
sn2007kf
sn2007kg
sn2007kh
sn2007kk
sn2007le
sn2007nq
sn2007ob
sn2007rx
sn2007ss
sn2007su
sn2007sw
sn2007ux
sn2008A
sn2008C
sn2008Q
sn2008Y
sn2008Z
sn2008ac
sn2008ad
sn2008ae
sn2008ai
sn2008ar
sn2008at
sn2008bi
sn2008bw
sn2008by
sn2008bz
sn2008cd
sn2008cf
sn2008cm
sn2008dr
sn2008ds
sn2008dt
sn2008fr
sn2008gb
sn2008gl
sn2008hj
sn2008hm
sn2008hs
sn2008hv
sn2009D
sn2009Y
sn2009ad
sn2009al
sn2009an
sn2009bv
sn2009dc
sn2009do
sn2009ds
sn2009fv
sn2009gf
sn2009hp
sn2009ig
sn2009jr
sn2009kk
sn2009kq
sn2009le
sn2009lf
sn2009li
sn2009na
sn2009nq
sn2010A
sn2010H
sn2010Y
sn2010ag
sn2010ai
sn2010cr
sn2010dt
sn2010dw
snf20080522000
snf20080522011
PTF10bjs


In [179]:
class spectrum:
    def __init__(self, filename, events):
        self.filename = filename
        print ("Creating: ", self.filename)
        self.wavelength, self.flux, self.flux_error = np.loadtxt(self.filename, unpack = True)
        #self.wavelength= np.loadtxt(self.filename, unpack = True)
        self.z = find_z(get_SN_name(self.filename))
        self.event = self.find_match(events)
        self.update_event_spectra_list()
        self.deredwavelength = self.cderedshift()
    
    
    def __eq__(self, other):
        return self.filename == other.filename
    
    def find_match(self, events):   #how to initialize this as self.event? 
        match_list = []
        for a in events:
            if a.name in self.filename:
                ev = a
                print (ev.name)
                return ev
        
    def update_event_spectra_list(self):
        if self not in self.event.spectra:
            self.event.spectra.append(self)
        else:
            i = self.event.spectra.index(self)
            self.event.spectra[i] = self
    
    def snr(self):
        temp_flux = gsmooth(self.wavelength, self.flux, self.flux_error, .001)
        snr = np.mean(temp_flux / self.flux_error)
        return snr
    
    def csm_flux(self):
        if self.snr() < 5:
            vexp_auto = .0025
        elif 5 <= self.snr() < 20:
            vexp_auto = .002
        elif 20 <= self.snr() < 40:
            vexp_auto = .0015
        elif 40 <= self.snr() < 60:
            vexp_auto = .001
        elif 60 <= self.snr() < 100:
            vexp_auto = .00075
        else:
            vexp_auto = .0005
        
        sm_flux = gsmooth(self.wavelength, self.flux, self.flux_error, vexp_auto)
        return sm_flux
    
    def cderedshift(self):
        cderedwavelength = self.wavelength / (1 + self.z)
        return cderedwavelength
    
    def min_wave(self, error = False, w1 = 5900, w2 = 6500, n = 1000):
        wave_domain = np.where((self.deredwavelength > w1) & (self.deredwavelength < w2)) #set domain as within minwave and maxwave
        list(wave_domain)
        flux_range = self.csm_flux()[wave_domain] #find corresponding flux vales for wavelengths within domain
        elem_flux = min(flux_range) #find minimum value within these flux vales to locate "dip"
        min_wave = self.deredwavelength[np.where(self.csm_flux() == elem_flux)][0] #find the corresponding wavelength
        
        local_flux_error = self.flux_error[wave_domain]
        local_median_error = np.median(local_flux_error)
        
        if error == True:
            flux_dist = []
            for i in range(n):
                point_sample = np.random.normal(elem_flux, local_median_error)
                flux_dist.append(point_sample)
        
            lower68bound = sorted(flux_dist)[int(round(n * .16)) - 1]
            upper68bound = sorted(flux_dist)[int(round(n * .84)) - 1]
            sigma = local_median_error
            return min_wave, sigma, lower68bound, upper68bound
        
        else:
            return min_wave
    
    def max_wave(self, error = False, w1 = 5900, w2 = 6500, n = 1000):
        wave_domain = np.where((self.deredwavelength > w1) & (self.deredwavelength < w2)) #set domain as within minwave and maxwave
        list(wave_domain)
        flux_range = self.csm_flux()[wave_domain] #find corresponding flux vales for wavelengths within domain
        elem_flux = max(flux_range) #find minimum value within these flux vales to locate "dip"
        max_wave = self.deredwavelength[np.where(self.csm_flux() == elem_flux)][0] #find the corresponding wavelength
        
        local_flux_error = self.flux_error[wave_domain]
        local_median_error = np.median(local_flux_error)
        
        if error == True:
            flux_dist = []
            for i in range(n):
                point_sample = np.random.normal(elem_flux, local_median_error)
                flux_dist.append(point_sample)
        
            lower68bound = sorted(flux_dist)[int(round(n * .16)) - 1]
            upper68bound = sorted(flux_dist)[int(round(n * .84)) - 1]
            sigma = local_median_error
            return max_wave, sigma, lower68bound, upper68bound
        
        else:
            return max_wave
    
    def eq_width(self):
        max_1 = self.max_wave(False, 5800, 6100)
        max_2 = self.max_wave(False, 6250, 6450)
        
        domain = np.where((self.deredwavelength >= max_1) & (self.deredwavelength <= max_2))
        list(domain)
        wave_range = self.deredwavelength[domain]
        flux_range = self.flux[domain]
        
        line_elem = np.polyfit([max_1, max_2], [self.csm_flux()[np.where(self.deredwavelength == max_1)],
                                                self.csm_flux()[np.where(self.deredwavelength == max_2)]], 1)
        #print(line_elem)
        line = line_elem[0] * wave_range + line_elem[1]
        #ask about this
        
        norm = flux_range / line
        new_norm = norm * 0 + 0.3
        #print (new_norm)
        

        a_curve = trapz((1 - new_norm), x = wave_range)
        a_line = max(wave_range) - min(wave_range)
        
        eq_width = a_line - a_curve
        
        #plt.plot(norm, color = 'black')
        #plt.plot(wave_range, norm, color = "orange")
        #plt.plot(wave_range , 1, color = "yellow")
        #plt.axvline (x=max_1, color = 'red')
        #plt.axvline (x=max_2, color = 'blue')
    
        return eq_width
     
    def si_ratio(self):
        pass
        
    def velocity(self, rest_wave = 6355):
        wave_prop = self.min_wave() / rest_wave
        ejecta_velocity = ejecta_velocity = c * (wave_prop ** 2 - 1) / (wave_prop ** 2 + 1)
        return ejecta_velocity
  
    def sigma_v(self, n = 1000, w1 = 5900, w2 = 6500, rest_wave = 6355):
        vel_dist = []
        if self.snr() < 5:
            vexp_auto = .0025
        elif 5 <= self.snr() < 20:
            vexp_auto = .002
        elif 20 <= self.snr() < 40:
            vexp_auto = .0015
        elif 40 <= self.snr() < 60:
            vexp_auto = .001
        elif 60 <= self.snr() < 100:
            vexp_auto = .00075
        else:
            vexp_auto = .0005
            
        for i in range(n):
            vexp_sample = np.random.normal(vexp_auto, vexp_auto / 4)
            flux_sample = gsmooth(self.deredwavelength, self.flux, self.flux_error, vexp_sample)
            
            wave_domain = np.where((self.deredwavelength > w1) & (self.deredwavelength < w2)) #set domain as within minwave and maxwave
            list(wave_domain)
            flux_range = flux_sample[wave_domain] #find corresponding flux vales for wavelengths within domain
            elem_flux = min(flux_range) #find minimum value within these flux vales to locate "dip"
            min_wave = self.deredwavelength[np.where(flux_sample == elem_flux)][0] #find the corresponding wavelength
        
            wavelength_prop = min_wave / rest_wave
            ejecta_velocity = c * (wavelength_prop ** 2 - 1) / (wavelength_prop ** 2 + 1)
            vel_dist.append(ejecta_velocity)
            
        lower68bound = sorted(vel_dist)[int(round(n * .16)) - 1]
        upper68bound = sorted(vel_dist)[int(round(n * .84)) - 1]
    
        return lower68bound, upper68bound, vel_dist
    
    def t_obs(self):
        file = self.filename
        date = file.split('-')[1]
        datetime = date[0:4] + "-" + date[4:6] + "-" + date[6:]
        t = Time(datetime)
        t.format = 'mjd'    
        t_obs = t.value
        return t_obs
    
    def t_max(self):
        t_max = peak_mjd_dict.get(self.event.name[2:], "None")
        return t_max
    
    def phase(self):
        phase = (self.t_obs() - self.t_max()) / (1 + self.z)
        return phase
    
    def v_0(self):
        v = self.velocity()
        t = self.phase()
        
        v_0 = (v + (0.285 * t)) / (1 - (0.0322 * t))
        #sigma_v_0 = ((self.sigma_v() / (1 - 0.0322 * t)) ** 2 + 0.22 ** 2) ** (1/2)
        return v_0#, sigma_v_0
    
    #def v_0_best(self):
        #data = self.event.spectra
        #v_0_list= []
        
        #for a in data:
        #temp_object = spectrum(data[0], SN_object_list)
            #v_0_each = temp_object.v_0()
            #v_0_list.append(v_0_each)
            
        #v_0_best = min(v_0_list)
        #return v_0_best
    
    
    def tot_ke(self):
        m = 2.86 #mass of the sun in 10 ** 30 kg)
        v = self.v_0() * 1000 #velocity in m/s
        ke = 0.5 * m * (v ** 2)
        return ke, "* 10^30 J"
    
    
    def intrinsic_color(self):
        B_maxMinusV_max_0 = -0.39 - (0.033 * self.v_0() / 1000)
        return B_maxMinusV_max_0

In [180]:
spectra = glob.glob("*.flm")
spectrum_object_list = []

for a in range(0, len(spectra)):    
    
    if get_SN_name(spectra[a]) not in SN_list:
        pass #associated supernova is not of CfA4
   
    elif len(np.loadtxt(spectra[a], unpack = True)) < 3:
        pass #spectrum is missing flux error
    
    else:
        spectrum_object = spectrum(spectra[a], SN_object_list)
        spectrum_object_list.append(spectrum_object)
        print(spectrum_object)

Creating:  sn2006ct-20060626.277-ui.flm
sn2006ct
Creating:  sn2007A-20070110.14-fast.flm
sn2007A
Creating:  sn2007bj-20070524.423-ui.flm
sn2007bj
Creating:  sn2007hu-20070911.14-fast.flm
sn2007hu
Creating:  sn2007is-20070919.14-fast.flm
sn2007is
Creating:  sn2007jg-20070918-not.flm
sn2007jg
Creating:  sn2007jg-20070920.46-fast.flm
sn2007jg
Creating:  sn2007nq-20071031.24-fast.flm
sn2007nq
Creating:  sn2007ob-20071107.14-fast.flm
sn2007ob
Creating:  sn2007su-20071229.08-fast.flm
sn2007su
Creating:  sn2007sw-20080103.52-fast.flm
sn2007sw
Creating:  sn2008at-20080305.30-fast.flm
sn2008at
Creating:  sn2008dt-20080908.228-ui.flm
sn2008dt
Creating:  sn2008fr-20080930.422-ui.flm
sn2008fr
Creating:  sn2008fr-20081007.368-ui.flm
sn2008fr
Creating:  sn2008fr-20081022.393-opt.flm
sn2008fr
Creating:  sn2008hj-20081205.296-ui.flm
sn2008hj
Creating:  sn2008Y-20080209.41-fast.flm
sn2008Y


In [185]:
for a in spectrum_object_list:
    print (a.z, a.eq_width())

0.031463 144.84281064856413
0.017646 107.0382038547786
0.016708 185.30394174138485
0.035391 136.72226241101203
0.029678 118.63611731046046
0.037131 765.1578968732003
0.037131 150.95007284518556
0.045031 131.66307985121978
0.033927 163.36066279340798
0.027862 149.73702695498037
0.025244 137.2151409810736
0.035001 122.71292491504835
0.035391 181.3807537442376
0.049 134.98570066730224
0.049 132.12583412774086


ValueError: max() arg is an empty sequence

In [32]:
for object in spectrum_object_list:
    if object.t_max() != "None":
        print (object.t_max())

54366.023
54366.023
54396.664


In [69]:
print (spectrum_object_list[5].event.name)

sn2007jg
